In [1]:
import ipywidgets as widgets
from IPython.display import display
import warnings
import PyPDF2
import datetime
import functools

warnings.filterwarnings('ignore')

# Create the file upload widget
search_file_upload = widgets.FileUpload(
    accept='',  # Accepted file types (e.g., '.txt', '.pdf', '.jpg', '.csv', etc.); leave empty for all file types
    multiple=True  # Set to True if you want to allow multiple files to be uploaded
)

fileListOutputSEARCH = widgets.Output()

# Display the widget
display(search_file_upload, fileListOutputSEARCH)

ongFileUploadValsSEARCH = []

def on_value_changeSEARCH(change):
    global ongFileUploadValsSEARCH
    global search_file_upload
    
    if (ongFileUploadValsSEARCH ==  [] and list(search_file_upload.value) == []) or ongFileUploadValsSEARCH != list(search_file_upload.value):

        @fileListOutputSEARCH.capture()
        def file_list_output_render():
            global ongFileUploadValsSEARCH
            
            fileListOutput.clear_output()
            display(widgets.Label("Uploaded Files to Be Searched: "))
            
            if change != {}:

                # set search_file_upload to previous search_file_upload and current search_file_upload files
                updatedFileUploadVal = []
                for file in ongFileUploadValsSEARCH:
                    updatedFileUploadVal.append(file)

                for file in search_file_upload.value:
                    if file not in updatedFileUploadVal:
                        updatedFileUploadVal.append(file)

                # update ong_file_upload value
                ongFileUploadValsSEARCH = updatedFileUploadVal
            
                for file in ongFileUploadValsSEARCH:
                    # extract number of pages of uploaded files
                    
                    fileOpen = open(file['name'], 'rb')
                    reader = PyPDF2.PdfReader(fileOpen)
                    totalPgs = len(reader.pages)
                    remButton = widgets.Button(description='Remove')
                    
                    def remove_button_onclick(button, fileInfo=""):
                        global ongFileUploadValsSEARCH
                        # update search_file_upload.value to remove chosen file
                        newVal = []
                        for file in search_file_upload.value:
                            if file['name'] != fileInfo:
                                newVal.append(file)
                        ongFileUploadValsSEARCH = []
                        search_file_upload.value = tuple(newVal)
                        
                    remButton.on_click(functools.partial(remove_button_onclick, fileInfo=file['name']))
                    
                    display(widgets.Label("\n  file name: " + file['name'] + "  #pages: " + str(totalPgs)), remButton)

                    search_file_upload.value = tuple(updatedFileUploadVal)

        file_list_output_render()

search_file_upload.observe(on_value_changeSEARCH, names='value')

FileUpload(value=(), description='Upload', multiple=True)

Output()

In [2]:
# Create the file upload widget
examples_file_upload = widgets.FileUpload(
    accept='.txt',  # Accepted file types (e.g., '.txt', '.pdf', '.jpg', '.csv', etc.); leave empty for all file types
    multiple=True  # Set to True if you want to allow multiple files to be uploaded
)

fileListOutput = widgets.Output()

# Display the widget
display(examples_file_upload, fileListOutput)

ongFileUploadVals = []

def on_value_change(change):
    global ongFileUploadVals
    global examples_file_upload
    
    if (ongFileUploadVals ==  [] and list(examples_file_upload.value) == []) or ongFileUploadVals != list(examples_file_upload.value):

        @fileListOutput.capture()
        def file_list_output_render():
            global ongFileUploadVals
            
            fileListOutput.clear_output()
            display(widgets.Label("Uploaded Files for Examples: "))
            
            if change != {}:

                # set examples_file_upload to previous examples_file_upload and current examples_file_upload files
                updatedFileUploadVal = []
                for file in ongFileUploadVals:
                    updatedFileUploadVal.append(file)

                for file in examples_file_upload.value:
                    if file not in updatedFileUploadVal:
                        updatedFileUploadVal.append(file)

                # update ong_file_upload value
                ongFileUploadVals = updatedFileUploadVal
            
                for file in ongFileUploadVals:
                    # extract number of pages of uploaded files
                    
                    fileOpen = open(file['name'], 'rb')
                    reader = PyPDF2.PdfReader(fileOpen)
                    totalPgs = len(reader.pages)
                    remButton = widgets.Button(description='Remove')
                    
                    def remove_button_onclick(button, fileInfo=""):
                        global ongFileUploadVals
                        # update examples_file_upload.value to remove chosen file
                        newVal = []
                        for file in examples_file_upload.value:
                            if file['name'] != fileInfo:
                                newVal.append(file)
                        ongFileUploadVals = []
                        examples_file_upload.value = tuple(newVal)
                        
                    remButton.on_click(functools.partial(remove_button_onclick, fileInfo=file['name']))
                    
                    display(widgets.Label("\n  file name: " + file['name'] + "  #pages: " + str(totalPgs)), remButton)

                    examples_file_upload.value = tuple(updatedFileUploadVal)

        file_list_output_render()

examples_file_upload.observe(on_value_change, names='value')

FileUpload(value=(), description='Upload', multiple=True)

Output()

In [3]:
print(search_file_upload.value, "\n")
print(examples_file_upload.value)

() 

()
